In [197]:
# importing the necessary packages
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from googletrans import Translator
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import html
from Stories import *

options = Options()
options.add_argument("--headless")
options.add_argument("--incognito")
options.add_argument("--log-level=3")
driver = webdriver.Chrome(options=options)
translator = Translator()
analyser = SentimentIntensityAnalyzer()


def company_url(company):
    company_url = "https://www.zonebourse.com/recherche/?mots={}&RewriteLast=recherche&noredirect=1&type_recherche=0".format(
        company)
    return company_url


class Scrapper:
    # Initializer / Instance Attributes
    # 10/10/2020 search functionality down + annoying button

    def __init__(self, company):
        self.company = company

    def get_stories(self):
        zb_url = 'https://www.zonebourse.com'
        company = self.company.company
        print(company)

        link1 = company_url(company)

        print('--FINDING COMPANY')
        xpath_company = '//*[@id="ALNI0"]/tbody/tr[2]/td[3]'
        # try
        driver.get(link1)
        href = driver.find_element_by_xpath(xpath_company).get_attribute('innerHTML')
        href = href.split(' ')[1].split('href=')[-1].split('"')[1]
        link2 = zb_url + href + '/actualite/'

        print('--FINDING NEWS RELATED TO COMPANY')
        print(link2)

        driver.get(link2)
        xpath_box = '//*[@id="autocomplete_forum"]'         

        searchbox = driver.find_element_by_xpath(xpath_box)
        searchbox.send_keys(company)
        searchbox.send_keys(Keys.ENTER)
        time.sleep(.3)
        try:
            method = 'method_1'
            pd.hello()
            """article_links = ['//*[@id="ALNI0"]/tbody/tr[{}]/td'.format(i) for i in range(1, 3)]
            article_links = [
                driver.find_element_by_xpath(i).get_attribute('innerHTML').split('href=')[-1].split('><')[0].split('"')[1]
                for i in article_links
            ]
            article_times = ['//*[@id="ALNI0"]/tbody/tr[{}]/td/a/div[2]'.format(i) for i in range(1, 3)]
            article_times = [driver.find_element_by_xpath(i).get_attribute('innerHTML') for i in article_times]"""
        except:
            # if error go through possible links and check bold area
            method = 'method_2'
            article_times = []
            driver.get(link2)
            article_links = []
            for i in range(1,6):
                xpath_news = f'//*[@id="ALNI4"]/tbody/tr[{i}]/td[2]/a'
                news_link = driver.find_element_by_xpath(xpath_news)
                article_links.append(news_link.get_attribute('href'))
            print(article_links)

        article_contents = []
        for j in range(len(article_links)):
            if method == 'method_1':
                link3 = zb_url + article_links[j]
            elif method == 'method_2':
                link3 = article_links[j]
            print(f'--FINDING ARTICLE {j+1}')
            driver.get(link3)
            print(link3)
            try:
                article_contents += [
                    driver.find_element_by_xpath("//div[contains(@id, 'grantexto')]").text
                    ]
                date = driver.find_element_by_xpath('//*[@id="zbCenter"]/div/table[2]/tbody/tr[3]/td[1]/div[2]/div[1]/div[2]')
                date = date.text
                date = datetime.strptime(date, '%d/%m/%Y | %H:%M')
                article_times += [str(date)]
                
                
                print('--CONTENTS LOADED')

            except:
                print('CONTENTS NULL')
                article_contents += ['Empty']
                continue

        print('contents done ! ---> Moving on to sentiment analysis')

        sentiment = []
        for p in range(len(article_contents)):
            contents_txt = html.unescape(article_contents[p])
            contents_english = translator.translate(contents_txt).text
            score = analyser.polarity_scores(contents_english)
            sentiment += [score['compound']]

        story1 = Story(date=article_times[0], sentiment=sentiment[0], company=company,
                       title='to_be_done', content=article_contents[0])

        story2 = Story(date=article_times[1], sentiment=sentiment[1], company=company,
                       title='to_be_done', content=article_contents[1])
        return story1, story2

        # dataframe_company = pd.DataFrame(company_list)
        # dataframe_company['Article Times'] = article_times
        # dataframe_company['Article Contents'] = article_contents



        # find stories of a given company
        # check match in terms of date & sentiment
        # initialise story objects
        # output all stories for a given company that match in terms of time



In [198]:
class Company:
    # Initializer / Instance Attributes
    def __init__(self, company, trend):
        self.company = company
        self.trend = trend

In [199]:
Renault = Company('Renault', 'up')
scrapper = Scrapper(Renault)
stories = scrapper.get_stories()

Renault
--FINDING COMPANY
--FINDING NEWS RELATED TO COMPANY
https://www.zonebourse.com/cours/action/RENAULT-4688//actualite/
['https://www.zonebourse.com/cours/action/RENAULT-4688/actualite-broker/RENAULT-SA-JP-Morgan-maintient-sa-recommandation-a-l-achat-31556536/', 'https://www.zonebourse.com/actualite-bourse/Les-Bourses-europeennes-choisissent-l-optimisme-pour-la-derniere-seance-de-la--31555241/', 'https://www.zonebourse.com/actualite-bourse/Paris-rebondit-sous-l-effet-de-LVMH--31553092/', 'https://www.zonebourse.com/cours/action/RENAULT-4688/actualite-broker/RENAULT-SA-Goldman-Sachs-toujours-positif-31552192/', 'https://www.zonebourse.com/actualite-bourse/Paris-rebondit-entrainee-par-le-luxe--31551949/']
--FINDING ARTICLE 1
https://www.zonebourse.com/cours/action/RENAULT-4688/actualite-broker/RENAULT-SA-JP-Morgan-maintient-sa-recommandation-a-l-achat-31556536/
CONTENTS NULL
--FINDING ARTICLE 2
https://www.zonebourse.com/actualite-bourse/Les-Bourses-europeennes-choisissent-l-optimis

In [163]:
link = 'https://www.zonebourse.com/actualite-bourse/Paris-rebondit-sous-l-effet-de-LVMH--31553092/'
options = Options()
options.add_argument("--headless")
options.add_argument("--incognito")
options.add_argument("--log-level=3")
driver = webdriver.Chrome(options=options)

In [164]:
driver.get(link)

In [186]:
a = driver.find_element_by_xpath('//*[@id="zbCenter"]/div/table[2]/tbody/tr[3]/td[1]/div[2]/div[1]/div[2]').text

In [203]:
stories[0].date

datetime.datetime(2020, 10, 16, 19, 5)

In [188]:
from datetime import datetime

In [194]:
b = str(datetime.strptime(a, '%d/%m/%Y | %H:%M'))

In [196]:
b

'2020-10-16 09:58:00'